In [11]:
import pygame
import numpy as np
import random as rand
import time
import math
rotationDeviation = []
# changeSpin and changeDirection = momentum
# changeRotation and changeLocation = absolute
class CollisionError(Exception):
    def __init__(self,oldA,oldB,newA,newB,i):
        super().__init__(f'{oldA} is now {newA} and {oldB} is now {newB}. {i} iterations have passed')
        
class Unit(pygame.sprite.Sprite):
    numberOfUnits = 0
    def __init__(self,x,y,unit_height,unit_width,name,drawstuff=False):
        super().__init__()
        Unit.numberOfUnits += 1
        self.height = unit_height
        self.width = unit_width
        self.diameter = math.sqrt(self.height**2 + self.width**2)
        self.longest_side = max(self.height,self.width)
        self.__name__ = name
        
        self.center = [x,y] # main
        self.corners = [[x-self.width/2,y-self.height/2],[x+self.width/2,y-self.height/2],[x+self.width/2,y+self.height/2],[x-self.width/2,y+self.height/2]] # main
        self.left = min(self.corners[0][0],self.corners[1][0],self.corners[2][0],self.corners[3][0]) # main
        self.right = max(self.corners[0][0],self.corners[1][0],self.corners[2][0],self.corners[3][0]) # main
        self.up = min(self.corners[0][1],self.corners[1][1],self.corners[2][1],self.corners[3][1]) # main
        self.down = max(self.corners[0][1],self.corners[1][1],self.corners[2][1],self.corners[3][1]) # main        
        self.surface = pygame.Surface([self.diameter,self.diameter],pygame.SRCALPHA) # own
        self.surface_center = [self.diameter/2,self.diameter/2] # own
        self.angle = 0
        
        self.cornerszeroed = [[0,0],[self.width,0],[self.width,self.height],[0,self.height]] # own
        self.leftzeroed = min(self.cornerszeroed[0][0],self.cornerszeroed[1][0],self.cornerszeroed[2][0],self.cornerszeroed[3][0]) # own
        self.rightzeroed = max(self.cornerszeroed[0][0],self.cornerszeroed[1][0],self.cornerszeroed[2][0],self.cornerszeroed[3][0]) # own
        self.upzeroed = min(self.cornerszeroed[0][1],self.cornerszeroed[1][1],self.cornerszeroed[2][1],self.cornerszeroed[3][1]) # own
        self.downzeroed = max(self.cornerszeroed[0][1],self.cornerszeroed[1][1],self.cornerszeroed[2][1],self.cornerszeroed[3][1]) # own
        self.drawstuff = drawstuff
        
        # bbox / mask
        self.bbox = [self.leftzeroed,self.upzeroed,self.rightzeroed-self.leftzeroed+1,self.downzeroed-self.upzeroed+1] # own
        self.bboxcorners = [(corner[0] - self.leftzeroed,corner[1] - self.upzeroed) for corner in self.cornerszeroed]
        self.bboxSurf = pygame.Surface((self.bbox[2], self.bbox[3]),pygame.SRCALPHA)
        pygame.draw.polygon(self.bboxSurf,(255,255,255),(self.bboxcorners))
        self.mask = pygame.mask.from_surface(self.bboxSurf)
        
        #collision
        self.offset_x = self.center[0] - self.left
        self.offset_y = self.center[1] - self.up
        
        #physics
        self.mass = 10
        self.bounciness = 2
        self.stickiness = 150
        self.stick_counterY = 0
        self.stick_counterX = 0
        self.stick_counterSpin = 0
        self.min_stick_speed = 0
        self.forceX = 0 
        self.forceY = 0
        self.spin = 0
        self.acceleration = 2
        self.maxSpeed = 20
        self.deceleration = .5
        self.maxSpin = 10
        self.integrity = 1
        self.angle = 0

    def changeSpin(self,degrees):
        if self.spin < self.maxSpin and self.spin > -1 * self.maxSpin:
            self.spin += degrees
    def changeRotation(self,degrees):
        for cornerIndex in list(range(len(self.corners))):
            self.corners[cornerIndex] = rotator(self.corners[cornerIndex],self.center,degrees)
        rotationDeviation.append(((self.corners[0][0]-self.corners[2][0])**2+(self.corners[0][1]-self.corners[2][1])**2)**.5)
        self.angle += degrees
    # regular movement
    def changeLocation(self,changeInX,changeInY):
#         self.rect[0] += changeInX
#         self.rect[1] += changeInY
        for corner in self.corners:
            corner[0] += changeInX
            corner[1] += changeInY
        self.center[0] += changeInX
        self.center[1] += changeInY
    def changeDirection(self,newForceX,newForceY):
        if abs(self.forceX + newForceX) <= self.maxSpeed:
            self.forceX += newForceX  
        else:
            self.forceX = math.copysign(self.maxSpeed,newForceX)
        if abs(self.forceY + newForceY) <= self.maxSpeed:
            self.forceY += newForceY
        else:
            self.forceY = math.copysign(self.maxSpeed,newForceY)
    def draw(self):
        for corner1 in self.corners:
            for corner2 in self.corners:
                pygame.draw.line(screen,(255,255,255),corner1,corner2,1)   
    def update(self,doMomentum=True,doBlit=True):
        if doMomentum:
            momentum(self)
        self.left = min(self.corners[0][0],self.corners[1][0],self.corners[2][0],self.corners[3][0]) # main
        self.right = max(self.corners[0][0],self.corners[1][0],self.corners[2][0],self.corners[3][0]) # main
        self.up = min(self.corners[0][1],self.corners[1][1],self.corners[2][1],self.corners[3][1]) # main
        self.down = max(self.corners[0][1],self.corners[1][1],self.corners[2][1],self.corners[3][1]) # main        
        self.surface = pygame.Surface([self.diameter,self.diameter],pygame.SRCALPHA) # own
        self.surface_center = [self.diameter/2,self.diameter/2] # own
        self.cornerszeroed = [[corner[0]-self.center[0]+self.surface_center[0],corner[1]-self.center[1]+self.surface_center[1]] for corner in self.corners]
        self.leftzeroed = min(self.cornerszeroed[0][0],self.cornerszeroed[1][0],self.cornerszeroed[2][0],self.cornerszeroed[3][0]) # main
        self.rightzeroed = max(self.cornerszeroed[0][0],self.cornerszeroed[1][0],self.cornerszeroed[2][0],self.cornerszeroed[3][0]) # main
        self.upzeroed = min(self.cornerszeroed[0][1],self.cornerszeroed[1][1],self.cornerszeroed[2][1],self.cornerszeroed[3][1]) # main
        self.downzeroed = max(self.cornerszeroed[0][1],self.cornerszeroed[1][1],self.cornerszeroed[2][1],self.cornerszeroed[3][1]) # main
        self.bbox = [self.leftzeroed,self.upzeroed,self.rightzeroed-self.leftzeroed+1,self.downzeroed-self.upzeroed+1] # own
        self.mask_offset = [self.center[0]-self.diameter-self.left,self.center[1]-self.diameter-self.up]
        self.angle %= 360
        # bboxMask
        self.bboxSurf = pygame.Surface((self.bbox[2], self.bbox[3]),pygame.SRCALPHA)
        self.bboxcorners = [(corner[0] - self.leftzeroed,corner[1] - self.upzeroed) for corner in self.cornerszeroed]
        pygame.draw.polygon(self.bboxSurf,(255,255,255),(self.bboxcorners))
        self.mask = pygame.mask.from_surface(self.bboxSurf)
        
        # true drawings
        if self.drawstuff:
            self.surface.fill((100,0,0))
            pygame.draw.rect(self.surface,(0,255,255),self.bbox)
            screen.blit(self.bboxSurf,(500,500))
        pygame.draw.polygon(self.surface,(255,255,255),((self.cornerszeroed[0][0],self.cornerszeroed[0][1]),(self.cornerszeroed[1][0],self.cornerszeroed[1][1]),(self.cornerszeroed[2][0],self.cornerszeroed[2][1]),(self.cornerszeroed[3][0],self.cornerszeroed[3][1])))
        

        #collision
        self.offset_x = self.center[0] - self.left
        self.offset_y = self.center[1] - self.up
        
        # draw on screen
        if doBlit:
            screen.blit(self.surface,(self.center[0] - self.surface_center[0], self.center[1] - self.surface_center[1]))
        
# rotation
def rotator(point,pointOfRotation,degrees):
    newPoint = [0,0]
    point[0] -= pointOfRotation[0]
    point[1] -= pointOfRotation[1]
    newPoint[0] = point[0]*math.cos(math.radians(degrees)) - point[1] * math.sin(math.radians(degrees)) # use round() to read
    newPoint[1] = point[0]*math.sin(math.radians(degrees)) + point[1] * math.cos(math.radians(degrees)) # but very bad accuracy
    newPoint[1] += pointOfRotation[1]
    newPoint[0] += pointOfRotation[0]
    return newPoint
def momentum(*units):
    for unit in units:
        unit.changeLocation(unit.forceX,unit.forceY)
        if unit.forceX != 0:
            if abs(unit.forceX) < unit.deceleration:
                unit.forceX = 0
            else:
                unit.forceX = math.copysign(abs(unit.forceX) - unit.deceleration,unit.forceX)
        if unit.forceY != 0:
            if abs(unit.forceY) < unit.deceleration:
                unit.forceY = 0
            else:
                unit.forceY = math.copysign(abs(unit.forceY) - unit.deceleration,unit.forceY)
        unit.changeRotation(unit.spin)
        if unit.spin != 0:
            if abs(unit.spin) < unit.deceleration:
                unit.spin = 0
            else:
                unit.spin = math.copysign(abs(unit.spin) - unit.deceleration,unit.spin)



# simple only 1 moves no mass 
def permDead(unitA,unitB,overlap): # forces
    unitB.forceX = unitA.forceX = 0
    unitB.forceY = unitA.forceY = 0
    unitA.spin = unitB.spin = 0
def sticker(unitA,unitB,overlap): # forces, maxSpeed, stickiness, changeLocation, 
    stickA = unitA.stickiness * unitA.maxSpeed / 20
    stickB = unitB.stickiness * unitB.maxSpeed / 20
    momentum(unitA)
    AforceX = unitA.forceX
    AforceY = unitA.forceY
    unitA.changeLocation(-1*AforceX,-1*AforceY)
    if abs(stickA) + abs(stickB) < abs(unitA.forceX):
        unitA.forceX /= (abs(stickA) + abs(stickB))/2
    else:
        if unitA.stick_counterX > (abs(stickA) + abs(stickB))/2:
            if unitA.forceX != 0:
                unitA.forceX = math.copysign(1,unitA.forceX)
            unitA.stick_counterX = 0
        else:
            if unitA.forceX != 0:
                unitA.changeLocation(math.copysign(.5,-1*unitA.forceX),0)
            unitA.forceX = 0
            unitA.stick_counterX += 5
    if abs(stickA) + abs(stickB) < abs(unitA.forceY):
        unitA.forceY /= (abs(stickA) + abs(stickB))/2
    else:
        if unitA.stick_counterY > (abs(stickA) + abs(stickB))/2:
            if unitA.forceY != 0:
                unitA.forceY = math.copysign(1,unitA.forceY)
            unitA.stick_counterY = 0
        else:
            if unitA.forceY != 0:
                unitA.changeLocation(0,math.copysign(.5,-1*unitA.forceY))
            unitA.forceY = 0
            unitA.stick_counterY += 5
    if abs(stickA) + abs(stickB) > abs(unitA.spin):
        unitA.changeRotation(unitA.spin/(abs(stickA) + abs(stickB)))
    else:
        if unitA.stick_counterSpin > (abs(stickA) + abs(stickB)):
            unitA.changeRotation(math.copysign(1,unitA.spin))
            unitA.spin = 0
        else:
            unitA.stick_counterSpin += 1
    unitA.update(False)
    unitB.update(False)
def pushOut(unitA,unitB,overlap): # forces, integrity,center(offset), changeLocation, changeRotation
    inside = True
#     print('moving piece',unitA.__name__,'overlaps',unitB.__name__,'at',unitA.forceX,unitA.forceY)
    oldA = unitA.forceX,unitA.forceY,unitA.center
    oldB = unitB.forceX,unitB.forceY,unitB.center
    integrity = sum((unitA.integrity,unitB.integrity))/2
    if unitA.forceX - unitB.forceX != 0:
        if unitA.forceY - unitB.forceY != 0:
            ratio = -1*integrity*math.copysign(1,unitA.forceX - unitB.forceX),-1*math.copysign(1,unitA.forceY - unitB.forceY)
        else:
            ratio = -1*integrity*math.copysign(1,unitA.forceX - unitB.forceX),0
    elif unitA.forceY - unitB.forceY != 0:
        ratio = 0,-1*integrity*math.copysign(1,unitA.forceY - unitB.forceY)   
    else:
        unitA.forceX = overlap[0]/unitA.offset_x * unitA.maxSpeed
        unitA.forceY = overlap[1]/unitA.offset_y * unitA.maxSpeed
#         unitB.forceX = overlap[0]/unitB.offset_x * unitB.maxSpeed
#         unitB.forceY = overlap[1]/unitB.offset_y * unitB.maxSpeed
        unitA.update(True,False)
        unitB.update(True,False)
        inside = False
    unitA.forceX = unitA.forceY = unitB.forceX = unitB.forceY = 0
    if unitA.spin - unitB.spin != 0:
        try:
            degrees = -1 * (unitA.spin - unitB.spin)/ (sum((ratio[0],ratio[1]))/2)
        except:
            degrees = -1 * math.copysign(1,unitA.spin-unitB.spin)
    else:
        degrees = 0
#     print('spin',unitA.spin)
#     print('degrees',degrees)
    unitA.spin = unitB.spin = 0
    i = 0
    while inside:   
        if i < 5000:
            if detectOverlap(unitA,unitB):
                unitA.changeLocation(ratio[0],ratio[1])
                unitA.changeRotation(degrees)
                unitA.update()
                unitB.update()
                i += 1
            else:
                inside = False
        else:
            inside = False
            unitA.forceX = overlap[0]/unitA.offset_x * unitA.maxSpeed
            unitA.forceY = overlap[1]/unitA.offset_y * unitA.maxSpeed
#             unitB.forceX = overlap[0]/unitB.offset_x * unitB.maxSpeed
#             unitB.forceY = overlap[1]/unitB.offset_y * unitB.maxSpeed
            unitA.update(True,False)
            unitB.update(True,False)
            raise CollisionError(oldA,oldB,(unitA.forceX,unitA.forceY),(unitB.forceX,unitB.forceY),i)
#     print('new speed is now',unitA.forceX,unitA.forceY)
#     print('done')
def throwOut(unitA,unitB,overlap): # forces, pushOut, bounciness
    # only for unitA movement assume unitB not moving
    bounciness = sum((unitA.bounciness,unitB.bounciness))/200
    if unitA.forceX != 0:
        if unitA.forceY != 0:
            ratio = -1*bounciness*unitA.forceX,-1*unitA.forceY
        else:
            ratio = -1*bounciness*unitA.forceX,0
    else:
        ratio = 0,-1*bounciness*unitA.forceY
    pushOut(unitA,unitB,overlap)
    unitA.forceX = ratio[0]
    unitA.forceY = ratio[1]
    unitA.update()
    unitB.update()

# complex 2 moves with mass
def bouncelessCollision(unitA,unitB,overlap): # forces, pushOut
    oldX = (unitA.forceX + unitB.forceX)/2
    oldY = (unitA.forceY + unitB.forceY)/2
    pushOut(unitA,unitB,overlap)
    unitA.forceX = unitB.forceX = oldX
    unitA.forceY = unitB.forceY = oldY
    unitA.update()
    unitB.update()
    
def permStick(unitA,unitB,overlap): # forces
    unitA.forceX = unitB.forceX = (unitA.forceX + unitB.forceX)/2
    unitA.forceY = unitB.forceY = (unitA.forceY + unitB.forceY)/2
    unitA.update()
    unitB.update()
    
def draggingCollision(unitA,unitB,overlap): # forces
    unitA.update()
    try:
        unitB.spin = -1 * unitA.spin/(unitA.mass-unitB.mass+1)
    except:
        unitB.spin = -1 * unitA.spin/(unitA.mass-unitB.mass)
    unitB.update()
    unitB.forceX = unitA.forceX = (unitA.forceX * unitA.mass - unitB.forceX * unitB.mass)/(unitA.mass + unitB.mass)
    unitB.forceY = unitA.forceY = (unitA.forceY * unitA.mass - unitB.forceY * unitB.mass)/(unitA.mass + unitB.mass)
    
def bouncing(unitA,unitB,overlap,overlap_area): 
    unitA.update()
    unitB.update()
    oldAX = unitA.forceX
    oldAY = unitA.forceY
    force = abs(oldAX) + abs(oldAY)
    overlap,overlpa_area = detectOverlap(unitA,unitB,True)
    
    pushOut(unitA,unitB,overlap)   
    unitA.forceX = overlap[0]/(unitA.diameter/2) * oldAX * -1
    unitA.forceY = overlap[1]/(unitA.diameter/2) * oldAY * -1
    
    if overlap_area%(unitA.width/2+1) or overlap_area%(unitA.height/2+1) == 0:
        print('divisible by 101')
        
        print('first attempt',oldAX,oldAY,'to',unitA.forceX,unitA.forceY)
        if overlap[1] > overlap[0]:
            print('horizontal collision')
            unitA.forceY = oldAY
            unitA.forceX = oldAX * -1          
        elif overlap[1] < overlap[0]:
            print('vertical collision')
            unitA.forceY = oldAY * -1
            unitA.forceX = oldAX  
        else:
            print('Broken?',overlap)
        print(overlap,'before:',oldAX,oldAY,'\tafter:',unitA.forceX,unitA.forceY)
#     else:
#         print('diagonal')
#         print(overlap,'before:',oldAX,oldAY,'\tafter:',unitA.forceX,unitA.forceY)
        
        
        
#     if force != abs(unitA.forceX) + abs(unitA.forceY):
#         print('straight on')
#         if abs(unitA.forceX) > abs(unitA.forceY):
#             unitA.forceX = math.copysign(force,unitA.forceX)
#         if abs(unitA.forceY) > abs(unitA.forceX):
#             unitA.forceY = math.copysign(force,unitA.forceY)
    
    unitA.update(True,False)
    unitB.update(True,False)
    # (-,0) = left
    # (+,0) = right
    # (0,-) = up
    # (0,+) = down
def unstableAccelerator(unitA,unitB,overlap): 
    oldAX = unitA.forceX
    oldAY = unitA.forceY
    force = abs(oldAX) + abs(oldAY)
    pushOut(unitA,unitB,overlap)
    integrity = sum((unitA.bounciness,unitB.bounciness))/2    
    unitA.forceX = overlap[0]/unitA.offset_x * oldAX
    unitA.forceY = overlap[1]/unitA.offset_y * oldAY
    print(force,abs(unitA.forceX)+abs(unitA.forceY))
    unitA.update()
    unitB.update()
def detectOverlap(unitA,unitB,showOverlap_area=False):
    offset_x = round((unitA.center[0] + unitA.offset_x) - (unitB.center[0] + unitB.offset_x))
    offset_y = round((unitA.center[1] + unitA.offset_y) - (unitB.center[1] + unitB.offset_y))
    overlap = unitA.mask.overlap(unitB.mask, (offset_x, offset_y))
    try:
        overlap_point = (unitA.surface_center[0]-overlap[0],unitA.surface_center[1]-overlap[1])
    except:
        if showOverlap_area:
            return None, None
        return None
    else:
        if showOverlap_area:
            print('overlap_area',unitA.mask.overlap_area(unitB.mask, (offset_x, offset_y)))
            return overlap_point,unitA.mask.overlap_area(unitB.mask, (offset_x, offset_y))
        else:
            return overlap_point
        
def drawAttributes(units):
    for unit in units:
        pygame.draw.line(unit.surface,(0,0,255),unit.surface_center,unit.surface_center) # blue surface_center
        for x in range(4):
            pygame.draw.line(screen,(0,255,0),unit.corners[x],unit.corners[x]) # green corners
            pygame.draw.line(unit.surface,(255,125,0),unit.cornerszeroed[x],unit.cornerszeroed[x]) # orange cornerszeroe
        pygame.draw.line(screen,(255,0,0),unit.center,unit.center) # red center
        unit.update(False)
# real object = drawing based on corners white
# self.surface = pygame.Surface; area on surface displayed based on absolute location dark red
# bbox = bounding box based on location of corners aqua
# self.maskSurf = mask display on main screen white
# self.cornerzeroed = green corners based on corners in relation to center
# movements based on center bc rotation

In [14]:
pygame.init()
clock = pygame.time.Clock()
screen = pygame.display.set_mode([800,800])
running = True
unit_height = 50
unit_width = 50
bob = Unit(100,100,100,100,'bob')
# bouncy = Unit(200,100,unit_height,unit_width)
# bob.stickiness = 1.1 # 1 - 2 = noticeable > 2 = stop instantly
# bouncy.stickiness = 1.1 # acceleration = speed in sticky max speed = time it takes to stop
# bouncy.bounciness = 15 # 1 - 20
# bouncy.mass = 100
a = Unit(500,200,unit_height*4,unit_width,'a')
# b = Unit(300,200,unit_height,unit_width,'b')
# c = Unit(200,250,unit_height,unit_width,'c')
# d = Unit(300,300,unit_height,unit_width,'d')
# e = Unit(200,350,unit_height,unit_width,'e')
movingList = [bob,a]
bob.drawstuff = True
# for x in movingList:
#     x.forceX = rand.randint(0,30)
#     x.forceY = rand.randint(0,30)
#     x.deceleration = 0
# bob.deceleration = 0
# bob.forceX = -20
# bob.forceY = -20
bob.bounciness = 100
wallA = Unit(400,795,10,805,'wallA')
wallB = Unit(400,0,10,805,'wallB')
wallC = Unit(0,400,805,10,'wallC')
wallD = Unit(795,400,805,10,'wallD')
nonmovingList = [wallA,wallB,wallC,wallD]
# wallA.drawstuff = True
drawnUnits = []
while running:
    if pygame.event.get([pygame.QUIT]):
        running = False
    if pygame.key.get_pressed():
        if pygame.key.get_pressed()[pygame.K_ESCAPE]:
            running = False
        if pygame.key.get_pressed()[pygame.K_SPACE]:
            print(wallA.cornerszeroed)
        if pygame.key.get_pressed()[pygame.K_q]:
            bob.changeRotation(1)
        if pygame.key.get_pressed()[pygame.K_e]:
            bob.changeRotation(-1)
        if pygame.key.get_pressed()[pygame.K_d]:
            bob.changeLocation(1,0)
        if pygame.key.get_pressed()[pygame.K_a]:
            bob.changeLocation(-1,0)
        if pygame.key.get_pressed()[pygame.K_w]:
            bob.changeLocation(0,-1)
        if pygame.key.get_pressed()[pygame.K_s]:
            bob.changeLocation(0,1)
        if pygame.key.get_pressed()[pygame.K_RIGHT]:
            bob.changeDirection(bob.acceleration,0)
        if pygame.key.get_pressed()[pygame.K_LEFT]:
            bob.changeDirection(-1 * bob.acceleration,0)
        if pygame.key.get_pressed()[pygame.K_UP]:
            bob.changeDirection(0,-1 * bob.acceleration)
        if pygame.key.get_pressed()[pygame.K_DOWN]:
            bob.changeDirection(0,bob.acceleration)
        if pygame.key.get_pressed()[pygame.K_p]:
            print(bob.center)
        if pygame.key.get_pressed()[pygame.K_z]:
            bob.changeSpin(2)
        if pygame.key.get_pressed()[pygame.K_c]:
            bob.changeSpin(-2)
        if pygame.key.get_pressed()[pygame.K_i]:
            wallA.changeLocation(0,-1)
        if pygame.key.get_pressed()[pygame.K_k]:
            wallA.changeLocation(0,1)
        if pygame.key.get_pressed()[pygame.K_j]:
            wallA.changeLocation(-1,0)
        if pygame.key.get_pressed()[pygame.K_l]:
            wallA.changeLocation(1,0)
        if pygame.key.get_pressed()[pygame.K_o]:
            print(bob.get_self())
    overlapList = []
    for x in movingList and nonmovingList:
        x.update(False)
    for k in movingList:
        for l in nonmovingList:
            overlap,overlap_area = detectOverlap(k,l,True)
            if overlap:
#                 print('overlap',overlap)
#                 pushOut(k,l,overlap) # double stacking crash
#                 throwOut(k,l,overlap) # hit corner? glitches
                bouncing(k,l,overlap,overlap_area) # fix glitches and crashes
    for i in movingList:
        for j in movingList:
            if i != j:
                overlap = detectOverlap(i,j)
                if overlap:
#                     overlapList.append(i)
#                     overlapList.append(j)
#                     print('overlap')
#                     i.forceX = 0
#                     i.forceY = 0
                    bouncelessCollision(i,j,overlap) # fix spin and double stacking
#                     permDead(i,j,overlap)    
#                     permStick(i,j,overlap)
#                     draggingCollision(i,j,overlap)
#                     sticker(i,j,overlap)
        
    screen.fill((0,0,0))
    for unit in nonmovingList:
        unit.update()
    
    for unit in movingList:
        unit.update()
    drawAttributes(drawnUnits)
    pygame.display.flip()    
    clock.tick(60)
pygame.quit()

overlap_area 101
overlap_area 1111
divisible by 101
first attempt -12.0 -10.5 to -2.7643895911751115 10.5
horizontal collision
(-16.289321881345245, 70.71067811865476) before: -12.0 -10.5 	after: 12.0 -10.5
overlap_area 808
overlap_area 1111
divisible by 101
first attempt 8.5 -11.0 to -8.5 -1.7562063326053168
vertical collision
(70.71067811865476, -11.289321881345245) before: 8.5 -11.0 	after: 8.5 11.0
overlap_area 86
overlap_area 617
divisible by 101
first attempt 14.5 0.0 to -9.16841486985343 -0.0
vertical collision
(44.710678118654755, 39.710678118654755) before: 14.5 0.0 	after: 14.5 -0.0
overlap_area 1100
overlap_area 1095
divisible by 101
first attempt 13.0 -0.0 to -7.852262632961933 0.0
horizontal collision
(42.710678118654755, 65.71067811865476) before: 13.0 -0.0 	after: -13.0 -0.0
overlap_area 151
overlap_area 611
divisible by 101
first attempt 19.5 0.0 to -10.95107901545464 -0.0
vertical collision
(39.710678118654755, 30.710678118654755) before: 19.5 0.0 	after: 19.5 -0.0
ove

divisible by 101
first attempt -7.0 -14.709331162702405 to 5.51507575950825 3.892220638362313
vertical collision
(55.710678118654755, 18.710678118654755) before: -7.0 -14.709331162702405 	after: -7.0 14.709331162702405


In [200]:
rotate([100,-150],[0,0],90) #-y,x

NameError: name 'rotate' is not defined

In [177]:
checkCollision('bob','joe','sam')

bob
joe
sam


In [ ]:
#cool spinning makes it smaller but not functional
import pygame
import numpy as np
import random as rand
import time
import math
class Unit(pygame.sprite.Sprite):
    numberOfUnits = 0
    def __init__(self,x,y,unit_width,unit_height):
        super().__init__()
        Unit.numberOfUnits += 1
        self.image = pygame.Surface([unit_width,unit_height])
        self.image.fill((255,255,255))
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.center = [self.rect.x + unit_width/2,self.rect.y+unit_height/2]
        self.corners = [[x,y],[x+unit_width,y],[x+unit_width,y+unit_height],[x,y+unit_height]]

    def rotate(self,degrees):
        print(self.corners)
        for corner in self.corners:
            old_corner = corner
            old_corner[0] -= self.center[0]
            old_corner[1] -= self.center[1]
#             old_corner[1] *= -1
#             x' = x*cos(theta) - y*sin(theta)
#             y' = x*sin(theta) + y*cos(theta)
            corner[0] = old_corner[0]*math.cos(math.radians(degrees)) - old_corner[1] * math.sin(math.radians(degrees))
            corner[1] = old_corner[0]*math.sin(math.radians(degrees)) + old_corner[1] * math.cos(math.radians(degrees))
            corner[0] += self.center[0]

            corner[1] += self.center[1]
            corner[1] = abs(corner[1])
#             old_corner[0] += self.center[0]
#             old_corner[1] += self.center[1]            
        print(self.corners)
    def changeLocation(self,changeInX,changeInY):
        self.rect.x += changeInX
        self.rect.y += changeInY
        for corner in self.corners:
            corner[0] += changeInX
            corner[1] += changeInY
        self.center[0] += changeInX
        self.center[1] += changeInY
    def draw(self):
        for corner1 in self.corners:
            for corner2 in self.corners:
                pygame.draw.line(screen,(255,255,255),corner1,corner2,1)   

In [3]:
old_corner = [-100,-100]
corner = [0,0]
corner[0] = round(old_corner[0]*math.cos(math.radians(90)) - old_corner[1] * math.sin(math.radians(90)))
corner[1] = round(old_corner[0]*math.sin(math.radians(90)) + old_corner[1] * math.cos(math.radians(90)))
print(corner)

[100, -100]


In [5]:
#looks 3-D but broken
import pygame
import numpy as np
import random as rand
import time
import math
class Unit(pygame.sprite.Sprite):
    numberOfUnits = 0
    def __init__(self,x,y,unit_width,unit_height):
        super().__init__()
        Unit.numberOfUnits += 1
        self.image = pygame.Surface([unit_width,unit_height])
        self.image.fill((255,255,255))
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.center = [self.rect[0] + unit_width/2,self.rect[1]+unit_height/2]
        self.corners = [[x,y],[x+unit_width,y],[x+unit_width,y+unit_height],[x,y+unit_height]]

    def rotate(self,degrees):
        print(self.corners)
        print(self.center)
        for corner in self.corners:
            old_corner = corner
            old_corner[0] -= self.center[0]
            old_corner[1] = math.copysign(abs(old_corner[1]) - abs(self.center[1]),self.center[1]-old_corner[1])
#             x' = x*cos(theta) - y*sin(theta)
#             y' = x*sin(theta) + y*cos(theta)
            print('modified',old_corner)
            corner[0] = round(old_corner[0]*math.cos(math.radians(degrees)) - old_corner[1] * math.sin(math.radians(degrees)))
            corner[1] = round(old_corner[0]*math.sin(math.radians(degrees)) + old_corner[1] * math.cos(math.radians(degrees)))
            print('transformed',corner)
            corner[0] += self.center[0]
            if corner[1] < 0:
                corner[1] *= -1
                corner[1] += self.center[1]
#             if corner[1] > 0:
#                 corner[1] += self.center[1]
#             old_corner[0] += self.center[0]
#             old_corner[1] += self.center[1]            
        print(self.corners)
    def changeLocation(self,changeInX,changeInY):
        self.rect.x += changeInX
        self.rect.y += changeInY
        for corner in self.corners:
            corner[0] += changeInX
            corner[1] += changeInY
        self.center[0] += changeInX
        self.center[1] += changeInY
    def draw(self):
        for corner1 in self.corners:
            for corner2 in self.corners:
                pygame.draw.line(screen,(255,255,255),corner1,corner2,1)   

In [338]:
x = pygame.image.load('C://Users\JaydenKoh\Games\square.png')
x

<Surface(314x312x24 SW)>

In [18]:
import pygame as pg

# Transparent surfaces with a circle and a triangle.
circle_surface = pg.Surface((60, 60), pg.SRCALPHA)
pg.draw.circle(circle_surface, (30, 90, 200), (30, 30), 30)
triangle_surface = pg.Surface((60, 60), pg.SRCALPHA)
pg.draw.polygon(triangle_surface, (160, 250, 0), ((30, 0), (60, 60), (0, 60)))


def main():
    screen = pg.display.set_mode((640, 480))
    clock = pg.time.Clock()

    # Use `pygame.mask.from_surface` to get the masks.
    circle_mask = pg.mask.from_surface(circle_surface)
    triangle_mask = pg.mask.from_surface(triangle_surface)

    # Also create rects for the two images/surfaces.
    circle_rect = circle_surface.get_rect(center=(320, 240))
    triangle_rect = triangle_surface.get_rect(center=(0, 0))

    done = False

    while not done:
        for event in pg.event.get():
            if event.type == pg.QUIT:
                done = True
            elif event.type == pg.MOUSEMOTION:
                triangle_rect.center = event.pos
            elif pygame.key.get_pressed()[pygame.K_SPACE]:
                print('circle',circle_rect.center)
                print('triangle',triangle_rect.center)
            elif pygame.key.get_pressed()[pygame.K_w]:
                triangle_rect.center = (300,250)
#         Now calculate the offset between the rects.
        offset_x = triangle_rect.x - circle_rect.x
        offset_y = triangle_rect.y - circle_rect.y

        # And pass the offset to the `overlap` method of the mask.
        isInside = circle_mask.overlap(triangle_mask, (offset_x, offset_y)) #
        if isInside:
            print('The two masks overlap!', isInside)

        screen.fill((30, 30, 30))
        screen.blit(circle_surface, circle_rect)
        screen.blit(triangle_surface, triangle_rect)

        pg.display.flip()
        clock.tick(60)


if __name__ == '__main__':
    pg.init()
    main()
    pg.quit()

The two masks overlap! (1, 38)
The two masks overlap! (0, 33)
The two masks overlap! (0, 29)
The two masks overlap! (0, 25)
The two masks overlap! (1, 21)
The two masks overlap! (15, 4)
The two masks overlap! (15, 4)
The two masks overlap! (16, 3)
The two masks overlap! (16, 3)
The two masks overlap! (16, 3)
The two masks overlap! (1, 21)
The two masks overlap! (2, 19)
The two masks overlap! (2, 19)
The two masks overlap! (3, 16)
The two masks overlap! (4, 15)
The two masks overlap! (5, 13)
The two masks overlap! (7, 11)
The two masks overlap! (9, 9)
The two masks overlap! (10, 8)
The two masks overlap! (11, 7)
The two masks overlap! (13, 5)
The two masks overlap! (15, 4)
The two masks overlap! (19, 2)
The two masks overlap! (21, 1)
The two masks overlap! (21, 1)
The two masks overlap! (25, 0)
The two masks overlap! (25, 0)
The two masks overlap! (25, 0)
The two masks overlap! (25, 0)
The two masks overlap! (28, 0)
The two masks overlap! (35, 1)
The two masks overlap! (55, 16)
The two 

In [17]:
import pygame
import random
 
# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
 
SCREEN_WIDTH = 700
SCREEN_HEIGHT = 500
BALL_SIZE = 25
 
class Ball:
    """
    Class to keep track of a ball's location and vector.
    """
    def __init__(self):
        self.x = 0
        self.y = 0
        self.change_x = 0
        self.change_y = 0
 
 
def make_ball():
    """
    Function to make a new, random ball.
    """
    ball = Ball()
    # Starting position of the ball.
    # Take into account the ball size so we don't spawn on the edge.
    ball.x = random.randrange(BALL_SIZE, SCREEN_WIDTH - BALL_SIZE)
    ball.y = random.randrange(BALL_SIZE, SCREEN_HEIGHT - BALL_SIZE)
 
    # Speed and direction of rectangle
    ball.change_x = random.randrange(-2, 3)
    ball.change_y = random.randrange(-2, 3)
 
    return ball
 
def main():
    """
    This is our main program.
    """
    pygame.init()
 
    # Set the height and width of the screen
    size = [SCREEN_WIDTH, SCREEN_HEIGHT]
    screen = pygame.display.set_mode(size)
 
    pygame.display.set_caption("Bouncing Balls")
 
    # Loop until the user clicks the close button.
    done = False
 
    # Used to manage how fast the screen updates
    clock = pygame.time.Clock()
 
    ball_list = []
 
    ball = make_ball()
    ball_list.append(ball)
 
    # -------- Main Program Loop -----------
    while not done:
        # --- Event Processing
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True
            elif event.type == pygame.KEYDOWN:
                # Space bar! Spawn a new ball.
                if event.key == pygame.K_SPACE:
                    ball = make_ball()
                    ball_list.append(ball)
 
        # --- Logic
        for ball in ball_list:
            # Move the ball's center
            ball.x += ball.change_x
            ball.y += ball.change_y
 
            # Bounce the ball if needed
            if ball.y > SCREEN_HEIGHT - BALL_SIZE or ball.y < BALL_SIZE:
                ball.change_y *= -1
            if ball.x > SCREEN_WIDTH - BALL_SIZE or ball.x < BALL_SIZE:
                ball.change_x *= -1
 
        # --- Drawing
        # Set the screen background
        screen.fill(BLACK)
 
        # Draw the balls
        for ball in ball_list:
            pygame.draw.circle(screen, WHITE, [ball.x, ball.y], BALL_SIZE)
 
        # --- Wrap-up
        # Limit to 60 frames per second
        clock.tick(60)
 
        # Go ahead and update the screen with what we've drawn.
        pygame.display.flip()
 
    # Close everything down
    pygame.quit()
if __name__ == "__main__":
    main()